In [36]:
import requests, json, time

import numpy as np
#from numpy import loadtxt
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Flatten

print('imports done')

imports done


In [7]:
# Because the champion id's have spaces between them, we have to condense them
# to do 1-hot encoding such that every index at the resulting array corresponds
# to a champion

full_champ_json = requests.get('http://ddragon.leagueoflegends.com/cdn/10.16.1/data/en_US/champion.json') \
                        .json()['data']
champ_ids = []
for value in full_champ_json.values(): 
    champ_ids.append(int(value['key']))
champ_ids.sort()

mapping = [None] * (champ_ids[len(champ_ids)-1] + 1)

for i in range(0,len(champ_ids)):
    mapping[champ_ids[i]] = i

print(mapping)

[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, None, None, 45, None, 46, 47, None, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, None, None, 60, 61, 62, None, None, 63, None, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, None, None, 77, 78, 79, 80, None, None, None, 81, None, 82, 83, None, 84, 85, 86, 87, 88, 89, 90, None, None, 91, 92, 93, 94, 95, 96, None, 97, None, 98, 99, 100, 101, None, None, None, 102, 103, None, None, None, 104, None, 105, 106, None, 107, None, None, None, None, 108, 109, 110, None, 111, None, None, None, None, 112, None, None, None, 113, None, None, 114, None, None, None, 115, None, 116, 117, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 118, 119, 120, None, 

In [8]:
#returns the index given the champion id 
def get_mapping(i:int) -> int: 
    return(mapping[i])

print(get_mapping(777))

147


In [54]:
f = open('data.json')

data = json.load(f)
arr = data['data']
mat = np.array(arr)

np.random.shuffle(mat)

# 90/10 training/test split
endpoint = int(len(mat) * 0.9)
test = mat[endpoint:]
mat = mat[1:endpoint]

# splits features and labels for TRAIN set
features = mat[:, 0:10] #training_features (5000 x 10)
results = mat[:,10] #training_labels (5000 x 1)

# reshapes using 1-hot encoding
# resulting input array has dimension: 4653 x 10 x 150
shape = (features.shape[0], features.shape[1], len(champ_ids))
reshapedFeatures = np.zeros(shape)
for i in range(0,shape[0]): #from 0-4999
    for j in range(0,shape[1]): #from 0-9
        index = get_mapping(features[i][j]) 
        reshapedFeatures[i][j][index] = 1

# splits features and labels for TEST set
featureTest = test[:, 0:10]
resultTest = test[:, 10]

# reshapes using 1-hot encoding
# resulting input array has dimension: 518 x 10 x 150
shape = (featureTest.shape[0], featureTest.shape[1], len(champ_ids))
reshapedFeatureTest = np.zeros(shape)
for i in range(0,shape[0]): #from 0-4999
    for j in range(0,shape[1]): #from 0-9
        index = get_mapping(featureTest[i][j]) 
        reshapedFeatureTest[i][j][index] = 1

In [58]:
#Create Model
model = Sequential()
model.add(Dense(20, input_shape=(10,150), activation='relu'))
model.add(Flatten())
model.add(Dense(20, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [59]:
#compile
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [60]:
#fit model and evaluate
model.fit(reshapedFeatures, results, epochs=200, batch_size=32, validation_split = 0.2)

_, taccuracy = model.evaluate(reshapedFeatures, results)
print('Accuracy: %.2f' % (taccuracy*100))
_, accuracy = model.evaluate(reshapedFeatureTest, resultTest)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/200
117/117 [==============================] - 0s 2ms/step - loss: 0.6927 - accuracy: 0.5164 - val_loss: 0.6928 - val_accuracy: 0.5070
Epoch 2/200
117/117 [==============================] - 0s 1ms/step - loss: 0.6895 - accuracy: 0.5411 - val_loss: 0.6938 - val_accuracy: 0.4952
Epoch 3/200
117/117 [==============================] - 0s 1ms/step - loss: 0.6833 - accuracy: 0.5750 - val_loss: 0.6980 - val_accuracy: 0.4973
Epoch 4/200
117/117 [==============================] - 0s 1ms/step - loss: 0.6732 - accuracy: 0.5860 - val_loss: 0.7030 - val_accuracy: 0.4984
Epoch 5/200
117/117 [==============================] - 0s 1ms/step - loss: 0.6608 - accuracy: 0.6072 - val_loss: 0.7242 - val_accuracy: 0.4984
Epoch 6/200
117/117 [==============================] - 0s 1ms/step - loss: 0.6509 - accuracy: 0.6196 - val_loss: 0.7319 - val_accuracy: 0.4715
Epoch 7/200
117/117 [==============================] - 0s 1ms/step - loss: 0.6362 - accuracy: 0.6408 - val_loss: 0.7356 - val_accuracy: 0.4823

Epoch 58/200
117/117 [==============================] - 0s 1ms/step - loss: 0.0031 - accuracy: 1.0000 - val_loss: 4.4203 - val_accuracy: 0.4887
Epoch 59/200
117/117 [==============================] - 0s 1ms/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 4.4641 - val_accuracy: 0.4898
Epoch 60/200
117/117 [==============================] - 0s 1ms/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 4.5857 - val_accuracy: 0.4812
Epoch 61/200
117/117 [==============================] - 0s 1ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 4.5975 - val_accuracy: 0.4962
Epoch 62/200
117/117 [==============================] - 0s 1ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 4.6830 - val_accuracy: 0.4898
Epoch 63/200
117/117 [==============================] - 0s 1ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 4.7395 - val_accuracy: 0.4941
Epoch 64/200
117/117 [==============================] - 0s 1ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 4.7933 - val_accuracy:

117/117 [==============================] - 0s 1ms/step - loss: 3.6014e-05 - accuracy: 1.0000 - val_loss: 7.2866 - val_accuracy: 0.4823
Epoch 114/200
117/117 [==============================] - 0s 1ms/step - loss: 3.3757e-05 - accuracy: 1.0000 - val_loss: 7.3455 - val_accuracy: 0.4876
Epoch 115/200
117/117 [==============================] - 0s 1ms/step - loss: 3.0851e-05 - accuracy: 1.0000 - val_loss: 7.4022 - val_accuracy: 0.4898
Epoch 116/200
117/117 [==============================] - 0s 1ms/step - loss: 2.9729e-05 - accuracy: 1.0000 - val_loss: 7.4371 - val_accuracy: 0.4834
Epoch 117/200
117/117 [==============================] - 0s 1ms/step - loss: 2.7628e-05 - accuracy: 1.0000 - val_loss: 7.4880 - val_accuracy: 0.4834
Epoch 118/200
117/117 [==============================] - 0s 1ms/step - loss: 2.5600e-05 - accuracy: 1.0000 - val_loss: 7.5073 - val_accuracy: 0.4823
Epoch 119/200
117/117 [==============================] - 0s 1ms/step - loss: 2.3470e-05 - accuracy: 1.0000 - val_loss: 7

117/117 [==============================] - 0s 1ms/step - loss: 8.1798e-07 - accuracy: 1.0000 - val_loss: 9.6790 - val_accuracy: 0.4791
Epoch 169/200
117/117 [==============================] - 0s 1ms/step - loss: 7.7848e-07 - accuracy: 1.0000 - val_loss: 9.7198 - val_accuracy: 0.4801
Epoch 170/200
117/117 [==============================] - 0s 1ms/step - loss: 7.3988e-07 - accuracy: 1.0000 - val_loss: 9.7512 - val_accuracy: 0.4791
Epoch 171/200
117/117 [==============================] - 0s 1ms/step - loss: 6.6799e-07 - accuracy: 1.0000 - val_loss: 9.8038 - val_accuracy: 0.4791
Epoch 172/200
117/117 [==============================] - 0s 1ms/step - loss: 6.3339e-07 - accuracy: 1.0000 - val_loss: 9.8304 - val_accuracy: 0.4758
Epoch 173/200
117/117 [==============================] - 0s 1ms/step - loss: 5.9497e-07 - accuracy: 1.0000 - val_loss: 9.8780 - val_accuracy: 0.4780
Epoch 174/200
117/117 [==============================] - 0s 1ms/step - loss: 5.6377e-07 - accuracy: 1.0000 - val_loss: 9

In [61]:
model.summary()


Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 10, 20)            3020      
_________________________________________________________________
flatten_6 (Flatten)          (None, 200)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 20)                4020      
_________________________________________________________________
dense_21 (Dense)             (None, 20)                420       
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 21        
Total params: 7,481
Trainable params: 7,481
Non-trainable params: 0
_________________________________________________________________


In [12]:
#Checks if file exists, if not, save
import os.path
if os.path.isfile('champs_model.h5') is False:
    model.save('champs_model.h5')